<a href="https://colab.research.google.com/github/Hey-Omair/Hey-Omair/blob/main/cap_pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%%writefile app.py
from flask import Flask, request, jsonify
from flask_cors import CORS
import cv2
import numpy as np
import os
import time

app = Flask(__name__)
CORS(app)

# FIXED: Renamed function to avoid recursion
def process_frame(frame):
    """Analyze single frame for basketball detection"""
    h, w = frame.shape[:2]
    # Demo stats - REPLACE with your detection.py logic
    attempts = np.random.randint(0, 5)
    made = np.random.randint(0, attempts+1)
    miss = attempts - made
    return {'attempts': attempts, 'made': made, 'miss': miss}

HTML_PAGE = '''<!DOCTYPE html>
<html>
<head>
    <title>🏀 Live Basketball Analyzer</title>
    <style>
        *{margin:0;padding:0;box-sizing:border-box;}
        body{font-family:-apple-system,BlinkMacSystemFont,'Segoe UI',Roboto,sans-serif;background:linear-gradient(135deg,#667eea 0%,#764ba2 100%);min-height:100vh;display:flex;flex-direction:column;align-items:center;padding:20px;}
        h1{color:white;text-shadow:0 2px 10px rgba(0,0,0,0.3);margin-bottom:20px;font-size:2.5em;}
        #videoContainer{max-width:90vw;max-height:50vh;overflow:hidden;border-radius:20px;box-shadow:0 10px 30px rgba(0,0,0,0.3);background:#000;}
        #video{width:100%;height:100%;object-fit:cover;}
        #controls{margin:30px 0;}
        button{padding:15px 30px;font-size:18px;border:none;border-radius:50px;background:#4CAF50;color:white;cursor:pointer;transition:all 0.3s;box-shadow:0 5px 15px rgba(76,175,80,0.4);}
        button:hover{background:#45a049;transform:translateY(-2px);}
        button:active{transform:translateY(0);}
        #stats{background:white;padding:30px;border-radius:20px;box-shadow:0 10px 30px rgba(0,0,0,0.2);max-width:500px;width:100%;margin:20px 0;}
        .stat-row{display:grid;grid-template-columns:repeat(3,1fr);gap:15px;margin:20px 0;}
        .stat-card{background:linear-gradient(135deg,#4CAF50,#45a049);color:white;padding:20px;border-radius:15px;text-align:center;}
        .stat-number{font-size:2.5em;font-weight:bold;}
        .status{opacity:0.8;font-size:0.9em;}
        #connectionStatus{padding:10px 20px;border-radius:25px;background:rgba(255,255,255,0.2);color:white;margin:10px 0;}
    </style>
</head>
<body>
    <h1>🏀 Real-Time Shot Analyzer</h1>
    <div id="videoContainer">
        <video id="video" autoplay playsinline muted></video>
    </div>
    <div id="controls">
        <button id="analyzeBtn" onclick="toggleAnalysis()">▶️ Start Analysis</button>
    </div>
    <div id="connectionStatus">🔄 Connecting camera...</div>
    <div id="stats">
        <h3 style="text-align:center;color:#333;margin-bottom:20px;">📊 Live Statistics</h3>
        <div class="stat-row">
            <div class="stat-card">
                <div class="stat-number" id="attempts">0</div>
                <div class="status">Attempts</div>
            </div>
            <div class="stat-card">
                <div class="stat-number" id="made">0</div>
                <div class="status">✅ Made</div>
            </div>
            <div class="stat-card">
                <div class="stat-number" id="missed">0</div>
                <div class="status">❌ Missed</div>
            </div>
        </div>
    </div>

    <script>
        const video = document.getElementById('video');
        const analyzeBtn = document.getElementById('analyzeBtn');
        const statusDiv = document.getElementById('connectionStatus');
        let stream, analyzing = false, rafId;

        navigator.mediaDevices.getUserMedia({
            video: {
                facingMode: 'environment',
                width: {ideal: 1280},
                height: {ideal: 720}
            }
        }).then(s => {
            stream = s;
            video.srcObject = stream;
            statusDiv.textContent = '✅ Camera ready! Point at basketball hoop';
            statusDiv.style.background = 'rgba(76, 175, 80, 0.3)';
        }).catch(e => {
            statusDiv.textContent = '❌ Camera access denied';
            statusDiv.style.background = 'rgba(244, 67, 54, 0.3)';
        });

        function toggleAnalysis() {
            analyzing = !analyzing;
            if (analyzing) {
                analyzeBtn.textContent = '⏹️ Stop Analysis';
                analyzeBtn.style.background = '#f44336';
                requestAnimationFrame(analyzeFrame);
            } else {
                analyzeBtn.textContent = '▶️ Start Analysis';
                analyzeBtn.style.background = '#4CAF50';
                cancelAnimationFrame(rafId);
            }
        }

        function analyzeFrame() {
            if (!analyzing || video.readyState !== video.HAVE_ENOUGH_DATA) {
                rafId = requestAnimationFrame(analyzeFrame);
                return;
            }

            const canvas = document.createElement('canvas');
            const ctx = canvas.getContext('2d');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            ctx.drawImage(video, 0, 0);

            canvas.toBlob(blob => {
                const formData = new FormData();
                formData.append('frame', blob, 'frame.jpg');

                fetch('/analyze_frame', {
                    method: 'POST',
                    body: formData
                }).then(r => r.json()).then(data => {
                    if (data.success) {
                        document.getElementById('attempts').textContent = data.stats.attempts || 0;
                        document.getElementById('made').textContent = data.stats.made || 0;
                        document.getElementById('missed').textContent = data.stats.miss || 0;
                    }
                }).catch(console.error);
            }, 'image/jpeg', 0.8);

            rafId = requestAnimationFrame(analyzeFrame);
        }
    </script>
</body>
</html>'''

@app.route('/')
def index():
    return HTML_PAGE

@app.route('/analyze_frame', methods=['POST'])
def analyze_frame():
    global stats_data
    stats_data = {'attempts': 0, 'made': 0, 'miss': 0}

    try:
        file = request.files['frame']
        nparr = np.frombuffer(file.read(), np.uint8)
        frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

        # FIXED: Call correct function name
        stats = process_frame(frame)
        stats_data.update(stats)

        return jsonify({'success': True, 'stats': stats_data})
    except Exception as e:
        return jsonify({'success': False, 'error': str(e)})

stats_data = {'attempts': 0, 'made': 0, 'miss': 0}

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=False)



Overwriting app.py


In [5]:
!pip install flask pyngrok opencv-python-headless flask-cors numpy


In [6]:
from pyngrok import ngrok, conf
import getpass

print("🔑 Get FREE token: https://dashboard.ngrok.com/get-started/your-authtoken")
token = getpass.getpass("Paste token here: ")
conf.get_default().auth_token = token
print("✅ Ngrok ready!")


🔑 Get FREE token: https://dashboard.ngrok.com/get-started/your-authtoken
Paste token here: ··········
✅ Ngrok ready!


In [7]:
%%writefile app.py
# [YOUR CODE FROM PREVIOUS MESSAGE - DON'T CHANGE IT]


Overwriting app.py


In [11]:
# ===== Cell 4: PROPER Flask + Ngrok =====
from threading import Thread
import time
import requests
from pyngrok import ngrok

# Flask app runner
def run_flask():
    from app import app
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

print("🚀 Starting Flask...")
flask_thread = Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

# Wait for Flask to start
print("⏳ Waiting for server...")
for i in range(15):
    try:
        r = requests.get("http://localhost:5000", timeout=2)
        if r.status_code == 200:
            print("✅ Flask READY!")
            break
        print(f"Waiting... ({i+1}/15)")
    except:
        time.sleep(2)
else:
    print("❌ Flask failed!")
    raise Exception("Server not starting")

# Ngrok tunnel
public_url = ngrok.connect(5000)
print("\n🎯 LIVE BASKETBALL ANALYZER!")
print(f"📱 PHONE: {public_url}")
print(f"💻 LOCAL: http://localhost:5000")
print("\n✅ Open PHONE → Allow camera → Start Analysis!")


Exception in thread Thread-4 (run_flask):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-3059695809.py", line 9, in run_flask
ImportError: cannot import name 'app' from 'app' (/content/app.py)


🚀 Starting Flask...
⏳ Waiting for server...
❌ Flask failed!


Exception: Server not starting

In [10]:
# Test if app.py works locally
!python app.py &
import time; time.sleep(3)
!curl http://localhost:5000 || echo "❌ NOT WORKING"
!pkill -f app.py


curl: (7) Failed to connect to localhost port 5000 after 0 ms: Connection refused
❌ NOT WORKING


In [12]:
# 🔥 ONE-CELL BASKETBALL ANALYZER (WORKS 100%)
!pip install nest-asyncio pyngrok flask flask-cors opencv-python-headless

import nest_asyncio
nest_asyncio.apply()

from flask import Flask, request, jsonify
from flask_cors import CORS
import cv2
import numpy as np
import time
from pyngrok import ngrok
import getpass

# Get ngrok token
print("🔑 FREE NGROK: https://dashboard.ngrok.com/get-started/your-authtoken")
token = getpass.getpass("Paste token: ")
ngrok.set_auth_token(token)

app = Flask(__name__)
CORS(app)

def process_frame(frame):
    attempts = np.random.randint(0, 5)
    made = np.random.randint(0, attempts+1)
    miss = attempts - made
    return {'attempts': attempts, 'made': made, 'miss': miss}

stats_data = {'attempts': 0, 'made': 0, 'miss': 0}

@app.route('/')
def index():
    return '''
<!DOCTYPE html>
<html>
<head><title>🏀 Live Basketball</title>
<style>
body{font-family:Arial;text-align:center;background:#667eea;padding:20px;color:white;}
video{max-width:90%;border:3px solid #4CAF50;border-radius:10px;}
button{padding:15px 30px;font-size:18px;background:#4CAF50;color:white;border:none;border-radius:25px;cursor:pointer;}
#stats{background:white;color:#333;padding:20px;border-radius:15px;max-width:400px;margin:20px auto;}
</style></head>
<body>
<h1>🏀 Basketball Analyzer</h1>
<video id="video" width="640" height="480" autoplay playsinline muted></video><br>
<button onclick="toggle()">Start Analysis</button>
<div id="stats">
Attempts: <span id="a">0</span> | Made: <span id="m">0</span> | Miss: <span id="s">0</span>
</div>

<script>
let analyzing=false;
navigator.mediaDevices.getUserMedia({video:{facingMode:'environment'}})
.then(s=>document.getElementById('video').srcObject=s);

function toggle(){
    analyzing=!analyzing;
    if(analyzing) analyze();
}

async function analyze(){
    if(!analyzing) return;

    const canvas=document.createElement('canvas');
    const ctx=canvas.getContext('2d');
    canvas.width=640; canvas.height=480;
    ctx.drawImage(document.getElementById('video'),0,0);

    canvas.toBlob(async blob=>{
        const fd=new FormData();
        fd.append('frame',blob);
        try{
            const r=await fetch('/analyze_frame',{method:'POST',body:fd});
            const data=await r.json();
            if(data.success){
                document.getElementById('a').innerText=data.stats.attempts;
                document.getElementById('m').innerText=data.stats.made;
                document.getElementById('s').innerText=data.stats.miss;
            }
        }catch(e){}
    });

    setTimeout(analyze,500);
}
</script>
</body>
</html>
'''

@app.route('/analyze_frame', methods=['POST'])
def analyze_frame():
    global stats_data
    file = request.files['frame']
    nparr = np.frombuffer(file.read(), np.uint8)
    frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    stats = process_frame(frame)
    stats_data.update(stats)
    return jsonify({'success': True, 'stats': stats_data})

# START EVERYTHING
print("🚀 Starting...")
public_url = ngrok.connect(5000)
print(f"\n🎯 LIVE: {public_url}")
print("📱 Open on PHONE → Allow camera → Start Analysis!")
app.run(host='0.0.0.0', port=5000)


🔑 FREE NGROK: https://dashboard.ngrok.com/get-started/your-authtoken
Paste token: ··········
🚀 Starting...

🎯 LIVE: NgrokTunnel: "https://proautomation-subobtusely-kristina.ngrok-free.dev" -> "http://localhost:5000"
📱 Open on PHONE → Allow camera → Start Analysis!
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [12/Jan/2026 06:45:15] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Jan/2026 06:45:23] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Jan/2026 06:45:24] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [12/Jan/2026 06:45:31] "POST /analyze_frame HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Jan/2026 06:45:31] "POST /analyze_frame HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Jan/2026 06:45:32] "POST /analyze_frame HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Jan/2026 06:45:33] "POST /analyze_frame HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Jan/2026 06:45:33] "POST /analyze_frame HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Jan/2026 06:45: